# Creating Timelapses

```{contents}
:local:
:depth: 2
```

## Introduction

## Technical requirements

```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
```

```bash
jupyter lab
```

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geebook/blob/master/chapters/09_timelapse.ipynb)

```bash
!pip install pygis
```

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

## The map function

In [ ]:
myList = ee.List.sequence(1, 10)
print(myList.getInfo())

In [ ]:
def computeSquares(number):
    return ee.Number(number).pow(2)

squares = myList.map(computeSquares)
print(squares.getInfo())

In [ ]:
squares = myList.map(lambda number: ee.Number(number).pow(2))
print(squares.getInfo())

## Creating cloud-free composites

In [ ]:
Map = geemap.Map()
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', 'Netherlands'))

Map.addLayer(fc, {'color': 'ff000000'}, "Netherlands")
Map.centerObject(fc)
Map

In [ ]:
years = ee.List.sequence(2013, 2021)

In [ ]:
def yearly_image(year):

    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, "year")

    collection = (
        ee.ImageCollection('LANDSAT/LC08/C01/T1')
        .filterDate(start_date, end_date)
        .filterBounds(fc)
    )

    image = ee.Algorithms.Landsat.simpleComposite(collection).clipToCollection(fc)

    return image

In [ ]:
images = years.map(yearly_image)

In [ ]:
vis_params = {'bands': ['B5', 'B4', 'B3'], 'max': 128}
for index in range(0, 9):
    image = ee.Image(images.get(index))
    layer_name = "Year " + str(index + 2013)
    Map.addLayer(image, vis_params, layer_name)

## Creating image timeseries

In [ ]:
collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

In [ ]:
region = ee.Geometry.BBox(-122.5549, 37.6968, -122.3446, 37.8111)

In [ ]:
start_date = '2016-01-01'
end_date = '2021-12-31'

In [ ]:
images = geemap.create_timeseries(collection, start_date, end_date, Map.user_roi, frequency='year', reducer='median')

In [ ]:
images.getInfo()

In [ ]:
Map = geemap.Map()

# S2 = (
#     ee.ImageCollection('COPERNICUS/S2_SR')
#     .filterBounds(ee.Geometry.Point([-122.45, 37.75]))
#     .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
# )

vis_params = {"min": 0, "max": 4000, "bands": ["B8", "B4", "B3"]}

Map.addLayer(images, {}, "Sentinel-2", False)
Map.add_time_slider(images, vis_params, time_interval=2)
Map.centerObject(region)
Map

## Creating timelapses

### NAIP

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map

In [ ]:
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-99.1019, 47.1274, -99.0334, 47.1562)
    Map.addLayer(region)
    Map.centerObject(region)

In [ ]:
collection = geemap.naip_timeseries(region, start_year=2009, end_year=2020, RGBN=True)

In [ ]:
years = geemap.image_dates(collection, date_format='YYYY').getInfo()
print(years)

In [ ]:
size = len(years)
images = collection.toList(size)
for i in range(size):
    image = ee.Image(images.get(i))
    Map.addLayer(image, {'bands': ['N', 'R', 'G']}, years[i])
Map

In [ ]:
geemap.naip_timelapse(
    region, out_gif="naip.gif", bands=['N', 'R', 'G'], frames_per_second=3, title='NAIP Timelapse'
)

## Creating Landsat timelapses

### Adding animated text

In [ ]:
import geemap
import os

In [ ]:
geemap.show_youtube('fDnDVuM_Ke4')

### Update the geemap package

In [ ]:
# geemap.update_package()

### Add animated text to an existing GIF

In [ ]:
in_gif = os.path.abspath('../data/animation.gif')
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_gif = os.path.join(out_dir, 'output.gif')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
geemap.show_image(in_gif)

#### Add animated text to GIF

In [ ]:
geemap.add_text_to_gif(
    in_gif,
    out_gif,
    xy=('5%', '5%'),
    text_sequence=1984,
    font_size=30,
    font_color='#0000ff',
    duration=100,
)

In [ ]:
geemap.show_image(out_gif)

#### Add place name

In [ ]:
geemap.add_text_to_gif(
    out_gif, out_gif, xy=('30%', '85%'), text_sequence="Las Vegas", font_color='black'
)

In [ ]:
geemap.show_image(out_gif)

#### Change font type

In [ ]:
geemap.system_fonts()

In [ ]:
geemap.add_text_to_gif(
    in_gif,
    out_gif,
    xy=('5%', '5%'),
    text_sequence=1984,
    font_size=30,
    font_color='#0000ff',
    duration=100,
)
geemap.add_text_to_gif(
    out_gif,
    out_gif,
    xy=('30%', '85%'),
    text_sequence="Las Vegas",
    font_type="timesbd.ttf",
    font_size=30,
    font_color='black',
)
geemap.show_image(out_gif)

### Create GIF from Earth Engine data

#### Prepare for an ImageCollection

In [ ]:
import ee
import geemap

ee.Initialize()

# Define an area of interest geometry with a global non-polar extent.
aoi = ee.Geometry.Polygon(
    [[[-179.0, 78.0], [-179.0, -58.0], [179.0, -58.0], [179.0, 78.0]]], None, False
)

# Import hourly predicted temperature image collection for northern winter
# solstice. Note that predictions extend for 384 hours; limit the collection
# to the first 24 hours.
tempCol = (
    ee.ImageCollection('NOAA/GFS0P25')
    .filterDate('2018-12-22', '2018-12-23')
    .limit(24)
    .select('temperature_2m_above_ground')
)

# Define arguments for animation function parameters.
videoArgs = {
    'dimensions': 768,
    'region': aoi,
    'framesPerSecond': 10,
    'crs': 'EPSG:3857',
    'min': -40.0,
    'max': 35.0,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}

#### Save the GIF to local drive

In [ ]:
saved_gif = os.path.join(os.path.expanduser('~'), 'Downloads/temperature.gif')
geemap.download_ee_video(tempCol, videoArgs, saved_gif)

In [ ]:
geemap.show_image(saved_gif)

#### Generate an hourly text sequence

In [ ]:
text = [str(n).zfill(2) + ":00" for n in range(0, 24)]
print(text)

#### Add text to GIF

In [ ]:
out_gif = os.path.join(os.path.expanduser('~'), 'Downloads/output2.gif')

In [ ]:
geemap.add_text_to_gif(
    saved_gif,
    out_gif,
    xy=('3%', '5%'),
    text_sequence=text,
    font_size=30,
    font_color='#ffffff',
)

In [ ]:
geemap.add_text_to_gif(
    out_gif,
    out_gif,
    xy=('32%', '92%'),
    text_sequence='NOAA GFS Hourly Temperature',
    font_color='white',
)

In [ ]:
geemap.show_image(out_gif)

### Adding colorbar to an image

In [ ]:
import geemap
import os

In [ ]:
# geemap.update_package()

#### Download a GIF

In [ ]:
from geemap import *

In [ ]:
url = 'https://i.imgur.com/MSde1om.gif'
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
download_from_url(url, out_file_name='temp.gif', out_dir=out_dir)

In [ ]:
in_gif = os.path.join(out_dir, 'temp.gif')
show_image(in_gif)

#### Get image URLs

In [ ]:
noaa_logo = 'https://bit.ly/3ahJoMq'
ee_logo = 'https://i.imgur.com/Qbvacvm.jpg'

#### Set output GIF path

In [ ]:
out_gif = os.path.join(out_dir, 'output.gif')

#### Add images to GIF

In [ ]:
add_image_to_gif(
    in_gif, out_gif, in_image=noaa_logo, xy=('2%', '80%'), image_size=(80, 80)
)

In [ ]:
add_image_to_gif(
    out_gif, out_gif, in_image=ee_logo, xy=('13%', '79%'), image_size=(85, 85)
)

#### Display output GIF

In [ ]:
show_image(out_gif)

#### Create a colorbar

In [ ]:
width = 250
height = 30
palette = ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
labels = [-40, 35]
colorbar = create_colorbar(
    width=width,
    height=height,
    palette=palette,
    vertical=False,
    add_labels=True,
    font_size=20,
    labels=labels,
)

In [ ]:
show_image(colorbar)

#### Add colorbar to GIF

In [ ]:
add_image_to_gif(
    out_gif, out_gif, in_image=colorbar, xy=('69%', '89%'), image_size=(250, 250)
)

In [ ]:
show_image(out_gif)

### Timelapse fading

In [ ]:
import ee
import geemap

In [ ]:
url = "https://i.imgur.com/ZWSZC5z.gif"

In [ ]:
geemap.show_image(url, verbose=False)

In [ ]:
out_gif = "gif_fading.gif"

In [ ]:
geemap.gif_fading(url, out_gif, verbose=False)

In [ ]:
geemap.show_image(out_gif)

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map

In [ ]:
roi = ee.Geometry.Polygon(
    [
        [
            [-69.315491, -22.837104],
            [-69.315491, -22.751488],
            [-69.190006, -22.751488],
            [-69.190006, -22.837104],
            [-69.315491, -22.837104],
        ]
    ]
)

In [ ]:
Map.addLayer(roi, {}, "ROI")
Map.centerObject(roi)

In [ ]:
title = "Sierra Gorda copper mines, Chile"
out_gif = "timelapse.gif"

In [ ]:
geemap.landsat_timelapse(
    roi,
    out_gif,
    start_year=2004,
    end_year=2010,
    frames_per_second=1,
    title=title,
    fading=False,
)

In [ ]:
geemap.show_image(out_gif)

In [ ]:
out_fading_gif = "timelapse_fading.gif"

In [ ]:
geemap.landsat_timelapse(
    roi,
    out_fading_gif,
    start_year=2004,
    end_year=2010,
    frames_per_second=1,
    title=title,
    fading=True,
)

In [ ]:
geemap.show_image(out_fading_gif)

## Creating GOES timelapses

In [ ]:
import os
import ee
import geemap

In [ ]:
# geemap.update_package()

In [ ]:
m = geemap.ee_initialize()

In [ ]:
region = ee.Geometry.Polygon(
    [
        [
            [-159.5954379282731, 60.40883060191719],
            [-159.5954379282731, 24.517881970830725],
            [-114.2438754282731, 24.517881970830725],
            [-114.2438754282731, 60.40883060191719],
        ]
    ],
    None,
    False,
)

In [ ]:
start_date = "2021-10-24T14:00:00"
end_date = "2021-10-25T01:00:00"
data = "GOES-17"
scan = "full_disk"

In [ ]:
col = geemap.goes_timeseries(start_date, end_date, data, scan, region)

In [ ]:
visParams = {
    'bands': ['CMI_C02', 'CMI_GREEN', 'CMI_C01'],
    'min': 0,
    'max': 0.8,
    'dimensions': 700,
    'framesPerSecond': 9,
    'region': region,
    'crs': col.first().projection(),
}

In [ ]:
out_dir = os.path.expanduser("~/Downloads")
out_gif = os.path.join(out_dir, "goes_timelapse.gif")
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
geemap.download_ee_video(col, visParams, out_gif)

In [ ]:
timestamps = geemap.image_dates(col, date_format='YYYY-MM-dd HH:mm').getInfo()

In [ ]:
geemap.add_text_to_gif(
    out_gif,
    out_gif,
    xy=('3%', '3%'),
    text_sequence=timestamps,
    font_size=20,
    font_color='#ffffff',
)

In [ ]:
geemap.goes_timelapse(
    out_gif, start_date, end_date, data, scan, region, framesPerSecond=10
)

## Creating MODIS timelapses

## Summary

## References